导入需要的包 & 声明数据集存放路径

In [1]:
import torch
import torchvision
from torchvision.datasets import ImageFolder
from torch import nn
from torch.utils.tensorboard import SummaryWriter
train_dir = "./Data/train_data"
valid_dir = "./Data/valid_data"
writer = SummaryWriter("logs")

使用torchvision库的ImageFolder读取数据集->train，valid，test

In [2]:
dataset_train = ImageFolder(train_dir, transform=torchvision.transforms.ToTensor()) # 7000个
dataset_valid = ImageFolder(valid_dir, transform=torchvision.transforms.ToTensor()) # 2000个

dataset_train[0][0].shape

torch.Size([3, 64, 64])

使用DataLoader加载数据集

In [3]:
batchsize = 256
dataloader_train = torch.utils.data.DataLoader(dataset_train, batch_size=batchsize, shuffle=True, num_workers=0, drop_last=False)
dataloader_valid = torch.utils.data.DataLoader(dataset_valid, batch_size=batchsize, shuffle=True, num_workers=0, drop_last=False)

模型搭建:

4层conv2卷积（kernel=3）,padding=same

Faltten->(256x(64 * 8 * 8))

FC1->(256x128)

Droupout正则化

FC2->(256x10)

softmax->得到十个类别的概率

In [4]:
from module_yy import ZH_MNIST

训练参数设置

In [5]:
n_epochs = 25
learning_rate = 0.0028
total_train_step = 0    # 训练的次数
total_valid_step = 0    # 测试的次数

创建模型，并转换到GPU上训练

In [6]:
module = ZH_MNIST()
module = module.cuda()

优化器 & 损失函数

In [7]:
loss = nn.CrossEntropyLoss()
loss = loss.cuda()
optimizer = torch.optim.Adam(module.parameters(), lr=learning_rate)

开始训练 & 验证

In [8]:

for epoch in range(n_epochs):
    print(f"==========epoch_{epoch+1}===========")
    # 先进行训练
    total_train_loss = 0
    count = 0
    module.train(True)
    for data in dataloader_train:
        imgs, labels = data
        imgs = imgs.cuda()
        labels = labels.cuda()
        output = module(imgs)
        train_loss = loss(output, labels)
        # 将原来的梯度清零
        optimizer.zero_grad()
        # 反向传播
        train_loss.backward()
        optimizer.step()
        total_train_step += 1
        count += 1
        # 将训练时的损失记录到tensorboard
        writer.add_scalar("train_loss", train_loss.item(), total_train_step)
        total_train_loss += train_loss.item()
    print(f"average_train_loss: {total_train_loss / count}")    
    
    # 每轮训练结束后用验证集对模型进行评估
    total_valid_loss, total_accuracy = 0, 0
    # 使用评估模式，避免Droupout的影响
    module.eval()
    with torch.no_grad():   # 不进行优化
        for imgs, labels in dataloader_valid:
            imgs = imgs.cuda()
            labels = labels.cuda()
            output = module(imgs)
            total_valid_loss += loss(output, labels).item()
            accuracy = (output.argmax(1) == labels).sum()
            total_accuracy += accuracy
    valid_loss = total_valid_loss / len(dataset_valid)
    print(f"valid_loss: {valid_loss}")
    writer.add_scalar("valid_loss", valid_loss, epoch+1)
    valid_accuracy = total_accuracy / len(dataset_valid)
    print(f"验证集上的准确率: {valid_accuracy}")
    writer.add_scalar("valid_accuracy", valid_accuracy, epoch+1)
    # 保存模型
    if epoch == n_epochs-1:
        torch.save(module.state_dict(), "ZH_Module.pth")
        print("save success...")
writer.close()

==========epoch_1===========
average_train_loss: 2.296346970966884
valid_loss: 0.00907253909111023
验证集上的准确率: 0.14000000059604645
==========epoch_2===========
average_train_loss: 2.124115437269211
valid_loss: 0.008055217027664185
验证集上的准确率: 0.4480000138282776
==========epoch_3===========
average_train_loss: 1.9101424515247345
valid_loss: 0.007434189260005951
验证集上的准确率: 0.6045000553131104
==========epoch_4===========
average_train_loss: 1.7753223053046636
valid_loss: 0.007179305136203766
验证集上的准确率: 0.6685000061988831
==========epoch_5===========
average_train_loss: 1.7092725123677934
valid_loss: 0.006787734270095825
验证集上的准确率: 0.7660000324249268
==========epoch_6===========
average_train_loss: 1.6557028932230813
valid_loss: 0.006621342718601227
验证集上的准确率: 0.8110000491142273
==========epoch_7===========
average_train_loss: 1.6109897451741355
valid_loss: 0.00642856776714325
验证集上的准确率: 0.8580000400543213
==========epoch_8===========
average_train_loss: 1.5820407313959939
valid_loss: 0.00628203129